# Quiz 3

In [1]:
import turicreate

In [2]:
products = turicreate.SFrame('data/amazon_baby.sframe')

In [3]:
# Se ignoran poductos cuyo puntaje es 3
products = products[products['rating']!= 3]

In [4]:
# Conteo de palabras en cada review
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [5]:
# Sentiment es una variable binaria, 1 si raiting>=4, 0 si <3.
products['sentiment'] = products['rating'] >= 4

In [6]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love',
                  'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

## Pregunta 1. 
**Utilice .apply () para crear una nueva función con los recuentos de cada una de las `selected_words`**: En el Notebook de la clase, se creó una columna `word_count` con el recuento de palabras para cada reseña. La primera tarea es crear una nueva columna en los productos SFrame con los recuentos de cada `selected_word` anterior y, en el proceso, verá cómo se puede usar el método .apply () para crear nuevas columnas en los datos (nuestras características) y cómo usar una función de Python, ¡que es un concepto extremadamente útil de comprender!

In [7]:
# Crea nuevas columnas con cada palabra de selected_words que
# contienen el número de veces que aparecen en cada review.
for word in selected_words:
    select_word_count = lambda dic: dic[word] if word in dic else 0
    products[word] = products['word_count'].apply(select_word_count, dtype=int)

In [8]:
# Calcula el numero total de plabras en selected_words
total_words = {}
for word in selected_words:
    total_words[word]=products[word].sum()
total_words

{'awesome': 3892,
 'great': 55791,
 'fantastic': 1664,
 'amazing': 2628,
 'love': 41994,
 'horrible': 1110,
 'bad': 4183,
 'terrible': 1146,
 'awful': 687,
 'wow': 425,
 'hate': 1107}

In [9]:
# Imprime la palabra más y menos utilizada de select_words
print('max: ',max(total_words, key=total_words.get))
print('min: ',min(total_words, key=total_words.get))

max:  great
min:  wow


## Pregunta 2

2. **Cree un nuevo modelo de análisis de sentimientos usando solo las `selected_words` como características:** En el Jupyter Notebook de la clase, se usó el recuento para todas las palabras como características para nuestro clasificador de sentimientos. Ahora, solo usará las `selected_words`:

In [10]:
# Se dividen aleatoriamente los datos en un conjunto de entrenamiento (80%) 
# y un conjunto de prueba (20%).
train_data,test_data = products.random_split(.8,seed=0)

### Modelo `selected_words`:

Se crea un modelo de regresión logística para clasificar los productos según las palabras seleccionadas: `selected_words` para saber si sus críticas son **positivas** o **negativas**.

+ Con este enfoque, ordene los coeficientes aprendidos de acuerdo con la columna "valor" usando .sort(). De las 11 palabras de selected_words, ¿cuál obtuvo el peso más positivo? ¿Cuál obtuvo el peso más negativo? ¿Estos valores tienen sentido para ti?

In [11]:
# Modelo de clasificación logística con selected_word
selected_words_model = turicreate.logistic_classifier.create(
    train_data,target='sentiment',
    features=selected_words,
    validation_set=test_data,
    verbose=False)

In [12]:
# Examinando los coeficientes generados por el clasificador
coeff = selected_words_model.coefficients.sort('value', ascending=False)
coeff.print_rows(num_rows=12, num_columns=5)

+-------------+-------+-------+----------------------+----------------------+
|     name    | index | class |        value         |        stderr        |
+-------------+-------+-------+----------------------+----------------------+
|     love    |  None |   1   |  1.3592688669224757  | 0.028068300152099997 |
| (intercept) |  None |   1   |  1.3365913848878148  | 0.008929969787657532 |
|   awesome   |  None |   1   |  1.1335346660341103  | 0.08399643983187567  |
|   amazing   |  None |   1   |  1.1000933113659914  | 0.09954776260465965  |
|  fantastic  |  None |   1   |  0.8858047568813963  | 0.11167591293399713  |
|    great    |  None |   1   |  0.8630655001195999  | 0.018955052444380473 |
|     wow     |  None |   1   | -0.00953823606771224 |  0.1604641122471162  |
|     bad     |  None |   1   | -0.9914778800650894  | 0.03848428664699065  |
|     hate    |  None |   1   | -1.3484407222463402  | 0.07715698604297318  |
|    awful    |  None |   1   |  -2.05290820403138   |  0.100997

In [13]:
# Palabra con peso mas positivo
print("Positivo:",coeff['name'][0])
# Palabra con peso mas negativo
print("Negativo:",coeff['name'][-1])

Positivo: love
Negativo: horrible


## Pregunta 3

**Comparación de la precisión de diferentes modelos de análisis de sentimientos:** 
+ ¿Cuál es la precisión de `selected_words_model` en `test_data?` 
+ ¿Cuál fue la precisión del modelo `sentiment_model` que aprendimos usando todos los recuentos de palabras en el Cuaderno de Jupyter anterior de las conferencias?
+ ¿Cuál es el clasificador de clase mayoritario de precisión en esta tarea? 
+ ¿Cómo compara los diferentes modelos aprendidos con el enfoque de línea de base donde solo estamos prediciendo la clase mayoritaria? 

Sugerencia: discutimos el clasificador de clases mayoritarias en la conferencia, que simplemente predice que cada punto de datos es de la clase más común. Esta es la línea de base es algo que definitivamente queremos superar con los modelos que aprendemos de los datos.

### Modelo selected_words_model 

In [14]:
accuracy_selected_model = selected_words_model.evaluate(test_data)['accuracy']
accuracy_selected_model 

0.8463848186404036

### Modelo sentiment_model

In [15]:
sentiment_model = turicreate.logistic_classifier.create(
    train_data,                                                    
    target='sentiment',                                                    
    features=['word_count'],
    validation_set=test_data,
    verbose=False)

In [16]:
accuracy_sentiment_model = sentiment_model.evaluate(test_data)['accuracy']
accuracy_sentiment_model 

0.9176975738650012

### Clase mayoritaria

In [17]:
num = test_data[test_data['sentiment']==1].shape[0]
den = test_data.shape[0]
majority_class = num/den
majority_class

0.8400192169108815

## Pregunta 4

4. **Interpretación de la diferencia de rendimiento entre los modelos:** para comprender por qué el modelo con todos los recuentos de palabras funciona mejor que el modelo con solo las palabras_seleccionadas, ahora se examinarán las reseñas de un producto en particular.

+ Se investigara un producto llamado "Baby Trend Diaper Champ". (Este es un bote de basura para pañales de bebé sucios, que mantiene el olor contenido). Al igual que con las reseñas del juguete de jirafa en el video de la conferencia, debe seleccionar todas las reseñas donde el nombre del producto sea "Baby Trend Diaper Champ". Llamemos a esta tabla diaper_champ_reviews.

+ Nuevamente, al igual que en el video, use sentiment_model para predecir el sentimiento de cada reseña en diaper_champ_reviews y ordene los resultados de acuerdo con su "predicted_sentiment".

+ ¿Cuál es el "predicted_sentiment" de la revisión más positiva para "Baby Trend Diaper Champ" según el sentiment_model del Jupyter Notebook de la conferencia? 

+ Ahora use el modelo de palabras_seleccionadas que aprendió usando solo las palabras_seleccionadas para predecir la opinión más positiva que encontró anteriormente. 

### Model sentiment

In [18]:
# Se predicen los sentimientos con el modelo sentiment
products['predicted_sentiment'] = sentiment_model.predict(
    products,                                                      
    output_type = 'probability')

In [19]:
# Se selecciona el producto 'Baby Trend Diaper Champ'
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [20]:
# Se ordenan las críticas de acuerdo al sentimiento predicho de mayor a menor
diaper_champ_reviews = diaper_champ_reviews.sort(
    'predicted_sentiment', ascending=False)

In [21]:
# Se calula valor del más positivo de acuerdo al modelo sentiment.
diaper_champ_reviews.select_columns(['name', 'review', 'rating', 'predicted_sentiment']).head(1)

name,review,rating,predicted_sentiment
Baby Trend Diaper Champ,I read a review belowthat can explain exactly ...,4.0,0.9999999999895941


### Model selected words

In [22]:
# Se calcula el valor predicho con el modelo selected_words para el más positivo
# en el modelo sentiment.
value = selected_words_model.predict(diaper_champ_reviews[0], 
                                     output_type='probability')
value[0]

0.7919288370624551